In [1]:
import requests
import feedparser
import re
from datetime import datetime
import nest_asyncio
from telegram import Bot
from telegram import ReplyKeyboardMarkup, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackQueryHandler


# Включаем поддержку вложенных циклов событий
nest_asyncio.apply()


In [2]:
# Словарь регионов (можно расширить позже)
CITIES = {
    "Москва": "27612",
}

In [3]:
# Функция для создания клавиатуры
def get_keyboard():
    keyboard = [
        ["Старт", "Погода"],  # Две кнопки в одной строке
        ["Настройки"],        # Одна кнопка в отдельной строке
    ]
    return ReplyKeyboardMarkup(keyboard, resize_keyboard=True)

# Обработчик команды /start
async def start(update, context):
    reply_markup = get_keyboard()
    await update.message.reply_text("Выберите действие:", reply_markup=reply_markup)

In [4]:
# Функция для создания inline-клавиатуры
def get_inline_keyboard():
    keyboard = [
        [InlineKeyboardButton("Старт", callback_data="start")],
        [InlineKeyboardButton("Погода", callback_data="weather")],
    ]
    return InlineKeyboardMarkup(keyboard)

# Обработчик команды /start
async def start(update, context):
    reply_markup = get_inline_keyboard()
    await update.message.reply_text("Выберите действие:", reply_markup=reply_markup)

In [5]:
# Обработчик текстовых сообщений
async def handle_message(update, context):
    text = update.message.text
    
    if text == "Старт":
        await update.message.reply_text("Привет! Я бот для прогноза погоды.", reply_markup=get_reply_keyboard())
    elif text == "Погода":
        await update.message.reply_text("Здесь будет прогноз погоды.", reply_markup=get_reply_keyboard())


In [6]:
# Обработчик нажатий на inline-кнопки
async def button_callback(update, context):
    query = update.callback_query
    await query.answer()
    
    if query.data == "start":
        await query.edit_message_text("Привет! Я бот для прогноза погоды.")
    elif query.data == "weather":
        await query.edit_message_text("Здесь будет прогноз погоды.")

In [7]:
# Функция для отправки сообщения в Telegram
async def send_telegram_message(token, chat_id, message):
    bot = Bot(token=token)
    try:
        await bot.send_message(chat_id=chat_id, text=message, parse_mode="Markdown")
        print("Сообщение успешно отправлено!")
    except Exception as e:
        print(f"Ошибка при отправке сообщения: {e}")

In [8]:
# Функция для получения прогноза погоды из RSS
def get_weather_from_rss(city_code):
    rss_url = f"https://meteoinfo.ru/rss/forecasts/index.php?s={city_code}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
    }
    
    try:
        response = requests.get(rss_url, headers=headers)
        response.raise_for_status()  # Проверка на HTTP-ошибки
        
        # Принудительно парсим содержимое как XML
        feed = feedparser.parse(response.content)
        
        if feed.bozo:  # Проверка на ошибки при парсинге
            return f"Ошибка при парсинге RSS: {feed.bozo_exception}"
        
        # Название города из первого элемента
        city_name = feed.entries[0].title.split(",")[0]
        weather_data = [f"*Прогноз погоды от Гидрометцентра России* 🌡️", f"📅 *{city_name}*"]
        
        for entry in feed.entries:
            title = entry.get("title", "Заголовок отсутствует")
            description = entry.get("description", "Описание отсутствует")
            
            # Извлекаем только дату из заголовка
            date_match = re.search(r"\d{1,2} [а-я]+", title)
            date = date_match.group(0) if date_match else "Дата отсутствует"
            
            # Разбиваем описание на отдельные строки
            description_lines = description.split(". ")
            
            # Разделяем строку с давлением и осадками
            pressure_line = description_lines[3] if len(description_lines) > 3 else ""
            precipitation_match = re.search(r"Вероятность осадков (\d+%)", pressure_line)
            precipitation = precipitation_match.group(0) if precipitation_match else "Вероятность осадков отсутствует"
            pressure = pressure_line.replace(precipitation, "").strip()
            
            # Формируем сообщение с иконками
            weather_message = (
                f"🗓️ *{date}*\n"
                f"*{description_lines[0]}* 🔹\n"  # Описание погоды жирным
                f"🌡️ {description_lines[1]}\n"    # Температура
                f"🌬️ {description_lines[2]}\n"    # Ветер
                f"📊 {pressure}\n"                # Давление
                f"🌧️ {precipitation}"             # Осадки
            )
            weather_data.append(weather_message)
        
        # Добавляем дату обновления и ссылку "Подробнее"
        update_date = datetime.now().strftime("%d.%m.%Y в %H:%M")  # Текущее время
        link = feed.entries[0].get("link", "Ссылка отсутствует")   # Берем ссылку из первого элемента
        weather_data.append(f"🔄 Прогноз обновлён {update_date}")
        weather_data.append(f"[Подробнее]({link})")
        
        return "\n\n".join(weather_data)
    except requests.RequestException as e:
        return f"Ошибка при запросе: {e}"
    except Exception as e:
        return f"Неизвестная ошибка: {e}"

In [9]:
# 7991239059:AAFtDMISLWwRbaFiIUZEpJsAqUhqRZS62fo


# Основная функция
def main():
    application = Application.builder().token("7991239059:AAFtDMISLWwRbaFiIUZEpJsAqUhqRZS62fo").build()

    # Регистрация обработчиков
    application.add_handler(CommandHandler("start", start))
    application.add_handler(CallbackQueryHandler(button_callback))

    application.run_polling()

if __name__ == "__main__":
    main()

No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\ext\_utils\networkloop.py", line 115, in network_retry_loop
    if not await do_action():
           ^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\ext\_utils\networkloop.py", line 108, in do_action
    return action_cb_task.result()
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\asyncio\futures.py", line 203, in result
    raise self._exception.with_traceback(self._exception_tb)
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\asyncio\tasks.py", line 314, in __step_run_and_handle_result
    result = coro.send(None)
             ^^^^^^^^^^^^^^^
  File "c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\telegram\ext\_updater.py", line 335, in polling_action_cb
   

RuntimeError: Cannot close a running event loop